In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import re
import string
import ipywidgets as widgets
from IPython.display import display, clear_output

def extract_year_from_string(s):
    match = re.search(r'^\d{4}', s)
    if match:
        return int(match.group(0))
    else:
        return s.lower().replace(' ', '_')

df_raw = pd.read_csv('gdp_data.csv')
df_raw.columns = [extract_year_from_string(c) for c in df_raw.columns]
df_raw.drop(['series_code', 'country_code'], axis=1, inplace=True)

df = pd.melt(df_raw,\
        id_vars=['country_name', 'series_name'], var_name='year', value_name='estimate')
df['estimate'] = df['estimate'].replace('..', None).astype(float)


country1_dropdown = widgets.Dropdown(
    options=df['country_name'].sort_values().unique(),
    description='Country 1:',
    value='United States',
)

country2_dropdown = widgets.Dropdown(
    options=df['country_name'].sort_values().unique(),
    description='Country 2:',
    value='China',
)

gdp_type_dropdown = widgets.Dropdown(
    options=df['series_name'].dropna().unique(),
    description='GDP Type:',
)

relative_diff_checkbox = widgets.Checkbox(
    value=False,
    description='Plot Relative Difference',
    disabled=False
)

def update_plot(country1, country2, gdp_type, relative_diff):
    clear_output(wait=True)

    # Filter data for the selected countries, GDP type, and source
    df_country1 = df[(df['country_name'] == country1) & (df['series_name'] == gdp_type)].dropna(how='any').sort_values(by='year')
    df_country2 = df[(df['country_name'] == country2) & (df['series_name'] == gdp_type)].dropna(how='any').sort_values(by='year')

    df_to_plot = pd.concat([df_country1.set_index('year')['estimate']
                            ,df_country2.set_index('year')['estimate']],\
                              keys=[country1, country2], axis=1)\
                           .dropna(how='any')

    fig, ax = plt.subplots(figsize=(10, 6))

    if relative_diff:
        df_to_plot = df_to_plot.div(df_to_plot.iloc[0]).sub(1)
        ax.set_ylabel('∆ from first year')
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
    else:
        ax.set_ylabel('GDP Estimate')

    df_to_plot.plot(ax=ax)


    ax.set_xlabel('Year')
    ax.set_title(f'GDP Estimates Comparison: {country1} vs {country2}')
    ax.legend()
    ax.grid(True)
    plt.show()

interactive_plot = widgets.interactive(update_plot, country1=country1_dropdown, country2=country2_dropdown, gdp_type=gdp_type_dropdown, relative_diff=relative_diff_checkbox)

display(interactive_plot)


interactive(children=(Dropdown(description='Country 1:', index=253, options=('Afghanistan', 'Africa Eastern an…